In [1]:
import os

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [2]:
%%time
import json
import pandas as pd
import numpy as np
import cv2
from PIL import Image
#from tesserocr import PyTessBaseAPI, PSM, RIL, PT
import matplotlib.pyplot as plt

CPU times: user 445 ms, sys: 288 ms, total: 733 ms
Wall time: 2.31 s


In [3]:
%%time
import lib.utils as utils
#import lib.config as config
import lib.model as model
#import lib.roi as roi
#import lib.split_pages as spages

/mnt/shared/scratch/ikarunak/apps/conda/envs/lightfootcat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CPU times: user 1.46 s, sys: 917 ms, total: 2.38 s
Wall time: 6.12 s


In [4]:
image_path = "images/cropped/The_Lightfoot_Herbarium_05_cropped_1.jpg"
image_dir = "images/cropped/"
default_yaml = "./prompts/default.yaml"

In [5]:
images = sorted(utils.load_images(image_dir))[:2]

In [6]:
simple_prompt = "Parse the input image for the text and refactor it into a JSON structure"

In [7]:
%%time
qwen_model = model.QWEN_Model(prompt=default_yaml, max_new_tokens = 20000, batch_size=2)

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:38<00:00,  7.70s/it]


CPU times: user 2.25 s, sys: 11.4 s, total: 13.6 s
Wall time: 41.1 s


In [8]:
qwen_model.conversation

[{'role': 'system',
  'content': 'SETUP: \nYou are an expert in horticulture, plant taxonomy and botanical catalogues\n'},
 {'role': 'system',
  'content': 'CONTEXT: \nThe following images provided are snapshots of botanical catalogue pages from botanical books.  The pages contain information on species, where they were collected and in which folder they were stored.  The information is organised in order of plant taxonomy with the highest classification being family name.  Typically the page contain information seperated in one to two columns.  Family names are typically Capitalized and species names can be found italicized.  The contents of the folders are written under each species and sometimes found at the start of the page (mostly due to continuation from the previous page).\n'},
 {'role': 'system',
  'content': 'GOAL: \nYour task is to parse the given image into a JSON object that follows the provided rules, instructions and template.\n'},
 {'role': 'system',
  'content': 'INSTR

In [9]:
%%time
outputs = qwen_model.batch_infer(images, save=True, debug=True)

>>> Using: 
 	Maximum new tokens = 20000 
 	Batch size = 2 
 	save_path = None
>>> Batch 1 starting...
	Processing text prompts...
	Reading Images...
	Processing inputs...
	Moving inputs to gpu...
	Performing inference...
	Inference Finished
	Seperating Ids...
	Decoding Ids...
	Seperating Outputs...
	Outputs stored!
	== Saving Pairs For Batch 1 ==
>>> Saving text output for images/cropped/The_Lightfoot_Herbarium_05_cropped_1.jpg
>>>> Saving JSON...
>>>> Saving CSV...
>>> Saving text output for images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg
>>>> Saving JSON...
>>>> Saving CSV...
	== Saving Done For Batch 1  ==
CPU times: user 43.7 s, sys: 743 ms, total: 44.4 s
Wall time: 45.9 s


In [10]:
out = outputs[0][1]

In [11]:
print(out)

```json
{
    "metadata": {
        "division": "N/A",
        "page": "132"
    },
    "contents": [
        {
            "familyName": "Aceraceae",
            "species": [
                {
                    "speciesName": "Acer campestre L.",
                    "folders": [
                        {
                            "description": "1 folder. Acer campestre [TA]",
                            "citations": ["N/A"]
                        }
                    ]
                },
                {
                    "speciesName": "Acer pseudoplatanus L.",
                    "folders": [
                        {
                            "description": "2 folders. Folder 1. Acer Pseudo-Platanus [G]. i. \"Maple. Bulls: [Bulstrode] Park\" [JL] Folder 2. Acer Pseudo-Platanus [TA].",
                            "citations": ["N/A"]
                        }
                    ]
                }
            ]
        },
        {
            "familyName": "Amaranthace